In [1]:
from pyspark.sql import SparkSession, types, functions as F

spark = SparkSession.builder \
.master("local[*]") \
.config("spark.driver.memory","8g") \
.config("spark.jars", "../resources/postgres_jar/postgresql-42.5.1.jar") \
.appName('steamAPI') \
.getOrCreate()


/spark/spark-3.3.2-bin-hadoop3/bin/load-spark-env.sh: line 68: ps: command not found


23/03/18 11:19:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
spark

In [3]:
steamapi="../resources/data/steamapi/*"
df=spark.read.parquet(steamapi,header=True)

In [9]:
steamspy="../resources/data/steamSpy/*"
df_spy = spark.read.parquet(steamspy,header=True)

In [10]:
df_spy.printSchema()

root
 |-- appid: long (nullable = true)
 |-- name: string (nullable = true)
 |-- developer: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- score_rank: string (nullable = true)
 |-- positive: long (nullable = true)
 |-- negative: long (nullable = true)
 |-- userscore: long (nullable = true)
 |-- owners: string (nullable = true)
 |-- average_forever: long (nullable = true)
 |-- average_2weeks: long (nullable = true)
 |-- median_forever: long (nullable = true)
 |-- median_2weeks: long (nullable = true)
 |-- price: string (nullable = true)
 |-- initialprice: string (nullable = true)
 |-- discount: string (nullable = true)
 |-- ccu: long (nullable = true)



In [5]:
df.printSchema()

root
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- steam_appid: string (nullable = true)
 |-- required_age: string (nullable = true)
 |-- is_free: string (nullable = true)
 |-- detailed_description: string (nullable = true)
 |-- about_the_game: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- supported_languages: string (nullable = true)
 |-- header_image: string (nullable = true)
 |-- website: string (nullable = true)
 |-- pc_requirements: string (nullable = true)
 |-- mac_requirements: string (nullable = true)
 |-- linux_requirements: string (nullable = true)
 |-- developers: string (nullable = true)
 |-- publishers: string (nullable = true)
 |-- price_overview: string (nullable = true)
 |-- packages: string (nullable = true)
 |-- package_groups: string (nullable = true)
 |-- platforms: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- screenshots: string (nullable

In [6]:
df.count()

61678

In [21]:
import requests
import pandas as pd

In [15]:


url="https://store.steampowered.com/api/appdetails"
params = {"appids":570}

test=requests.get(url=url,params=params)

In [17]:
dota=test.json()

In [27]:
dota['570']['data']

{'type': 'game',
 'name': 'Dota 2',
 'steam_appid': 570,
 'required_age': 0,
 'is_free': True,
 'dlc': [1241930, 652720],
 'detailed_description': "<strong>The most-played game on Steam.</strong><br>Every day, millions of players worldwide enter battle as one of over a hundred Dota heroes. And no matter if it's their 10th hour of play or 1,000th, there's always something new to discover. With regular updates that ensure a constant evolution of gameplay, features, and heroes, Dota 2 has truly taken on a life of its own.<br><br><strong>One Battlefield. Infinite Possibilities.</strong><br>When it comes to diversity of heroes, abilities, and powerful items, Dota boasts an endless array—no two games are the same. Any hero can fill multiple roles, and there's an abundance of items to help meet the needs of each game. Dota doesn't provide limitations on how to play, it empowers you to express your own style.<br><br><strong>All heroes are free.</strong><br>Competitive balance is Dota's crown j

In [32]:
data_pandas = pd.DataFrame.from_dict(dota['570']['data'], orient='index')

In [35]:
data_pandas.dtypes

0    object
dtype: object

In [36]:


url="https://store.steampowered.com/api/appdetails"
params = {"appids":10}

test2=requests.get(url=url,params=params)
counter = test2.json()
counter['10']['data']

{'type': 'game',
 'name': 'Counter-Strike',
 'steam_appid': 10,
 'required_age': 0,
 'is_free': False,
 'detailed_description': "Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic missions. Take out enemy sites. Rescue hostages. Your role affects your team's success. Your team's success affects your role.",
 'about_the_game': "Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic missions. Take out enemy sites. Rescue hostages. Your role affects your team's success. Your team's success affects your role.",
 'short_description': "Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic mis

In [37]:
list_data = []
list_data.append(dota['570']['data'])
list_data.append(counter['10']['data'])
list_data

[{'type': 'game',
  'name': 'Dota 2',
  'steam_appid': 570,
  'required_age': 0,
  'is_free': True,
  'dlc': [1241930, 652720],
  'detailed_description': "<strong>The most-played game on Steam.</strong><br>Every day, millions of players worldwide enter battle as one of over a hundred Dota heroes. And no matter if it's their 10th hour of play or 1,000th, there's always something new to discover. With regular updates that ensure a constant evolution of gameplay, features, and heroes, Dota 2 has truly taken on a life of its own.<br><br><strong>One Battlefield. Infinite Possibilities.</strong><br>When it comes to diversity of heroes, abilities, and powerful items, Dota boasts an endless array—no two games are the same. Any hero can fill multiple roles, and there's an abundance of items to help meet the needs of each game. Dota doesn't provide limitations on how to play, it empowers you to express your own style.<br><br><strong>All heroes are free.</strong><br>Competitive balance is Dota's 

In [38]:
data_pandas = pd.DataFrame.from_dict(list_data, orient='columns')

In [39]:
data_pandas

,type,name,steam_appid,required_age,is_free,dlc,detailed_description,about_the_game,short_description,supported_languages,...,genres,screenshots,movies,recommendations,release_date,support_info,background,background_raw,content_descriptors,price_overview
0,game,Dota 2,570,0,True,"[1241930, 652720]",<strong>The most-played game on Steam.</strong...,<strong>The most-played game on Steam.</strong...,"Every day, millions of players worldwide enter...","Bulgarian, Czech, Danish, Dutch, English<stron...",...,"[{'id': '1', 'description': 'Action'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 256692021, 'name': 'Dota 2 - Join the ...",{'total': 14325},"{'coming_soon': False, 'date': '9 Jul, 2013'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",NaN
1,game,Counter-Strike,10,0,False,NaN,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,"English<strong>*</strong>, French<strong>*</st...",...,"[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 133083},"{'coming_soon': False, 'date': '1 Nov, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",https://cdn.akamai.steamstatic.com/steam/apps/...,https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [2, 5], 'notes': 'Includes intense vio...","{'currency': 'SGD', 'initial': 1000, 'final': ..."


In [40]:
data_pandas.dtypes

type                    object
name                    object
steam_appid              int64
required_age             int64
is_free                   bool
dlc                     object
detailed_description    object
about_the_game          object
short_description       object
supported_languages     object
reviews                 object
header_image            object
website                 object
pc_requirements         object
mac_requirements        object
linux_requirements      object
developers              object
publishers              object
packages                object
package_groups          object
platforms               object
metacritic              object
categories              object
genres                  object
screenshots             object
movies                  object
recommendations         object
release_date            object
support_info            object
background              object
background_raw          object
content_descriptors     object
price_ov

In [41]:
testing = data_pandas.astype(str)

In [42]:
testing.dtypes

type                    object
name                    object
steam_appid             object
required_age            object
is_free                 object
dlc                     object
detailed_description    object
about_the_game          object
short_description       object
supported_languages     object
reviews                 object
header_image            object
website                 object
pc_requirements         object
mac_requirements        object
linux_requirements      object
developers              object
publishers              object
packages                object
package_groups          object
platforms               object
metacritic              object
categories              object
genres                  object
screenshots             object
movies                  object
recommendations         object
release_date            object
support_info            object
background              object
background_raw          object
content_descriptors     object
price_ov

In [48]:
from pyspark.sql.types import *

schema = StructType([
    StructField("type", StringType(), True),
    StructField("name", StringType(), True),
    StructField("steam_appid", IntegerType(), True),
    StructField("required_age", StringType(), True),
    StructField("is_free", StringType(), True),
    StructField("dlc", StringType(), True),
    StructField("detailed_description", StringType(), True),
    StructField("about_the_game", StringType(), True),
    StructField("short_description", StringType(), True),
    StructField("supported_languages", StringType(), True),
    StructField("reviews", StringType(), True),
    StructField("header_image", StringType(), True),
    StructField("website", StringType(), True),
    StructField("pc_requirements", StringType(), True),
    StructField("mac_requirements", StringType(), True),
    StructField("linux_requirements", StringType(), True),
    StructField("developers", StringType(), True),
    StructField("publishers", StringType(), True),
    StructField("packages", StringType(), True),
    StructField("package_groups", StringType(), True),
    StructField("platforms", StringType(), True),
    StructField("metacritic", StringType(), True),
    StructField("categories", StringType(), True),
    StructField("genres", StringType(), True),
    StructField("screenshots", StringType(), True),
    StructField("movies", StringType(), True),
    StructField("recommendations", StringType(), True),
    StructField("release_date", StringType(), True),
    StructField("support_info", StringType(), True),
    StructField("background", StringType(), True),
    StructField("background_raw", StringType(), True),
    StructField("content_descriptors", StringType(), True),
    StructField("price_overview", StringType(), True)
])


In [51]:
schema=None

In [52]:
df_spark = spark.read.schema(schema).parquet(steamapi,header=True)

TypeError: schema should be StructType or string

In [50]:
df_spark.printSchema()

root
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- steam_appid: integer (nullable = true)
 |-- required_age: string (nullable = true)
 |-- is_free: string (nullable = true)
 |-- dlc: string (nullable = true)
 |-- detailed_description: string (nullable = true)
 |-- about_the_game: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- supported_languages: string (nullable = true)
 |-- reviews: string (nullable = true)
 |-- header_image: string (nullable = true)
 |-- website: string (nullable = true)
 |-- pc_requirements: string (nullable = true)
 |-- mac_requirements: string (nullable = true)
 |-- linux_requirements: string (nullable = true)
 |-- developers: string (nullable = true)
 |-- publishers: string (nullable = true)
 |-- packages: string (nullable = true)
 |-- package_groups: string (nullable = true)
 |-- platforms: string (nullable = true)
 |-- metacritic: string (nullable = true)
 |-- categories: string (nullable = true)
 

In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet("../resources/data/app_list/app_list_2023-03-18.parquet")

In [8]:
df.sort_values(['appid'],ascending=True)

,appid,name
33,10,Counter-Strike
135,20,Team Fortress Classic
125,30,Day of Defeat
102,40,Deathmatch Classic
157,50,Half-Life: Opposing Force
...,...,...
48182,2333530,Dinobot and Tiara Present: ApplePop
46877,2333720,Draft Day Sports: College Basketball 2023
51548,2335330,Uncle Kenny The Game
50901,2338600,Dark City: International Intrigue


In [7]:
df.dtypes

appid     int64
name     object
dtype: object

In [9]:
df = pd.read_parquet("../resources/data/app_list/app_list_2023-03-20.parquet")
df['appid'].count()

61774